In [1]:
!pip install tensorflow_text

In [1]:
from sklearn.metrics import pairwise

import tensorflow as tf
import tensorflow_hub as hub

import re
import tensorflow_text as text

import numpy as np

from naps.pipelines.read_naps import read_naps_dataset

In [2]:
preprocessor = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2")

encoder = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3",
    trainable=True)

In [3]:
def flatten(iterable):
    iterator, sentinel, stack = iter(iterable), object(), []
    while True:
        value = next(iterator, sentinel)
        if value is sentinel:
            if not stack:
                break
            iterator = stack.pop()
        elif isinstance(value, str):
            yield str(value)
        else:
            try:
                new_iterator = iter(value)
            except TypeError:
                yield value
            else:
                stack.append(iterator)
                iterator = new_iterator


# trainA, trainB, test = read_naps_dataset()
# for name, ds in zip(("trainA", "trainB", "test"), read_naps_dataset()):
#     print("DATASET %s" % name)
def get_unique_tokens():
    ds, _, test = read_naps_dataset()
    tokens_total = []
    with ds:
        
        for d in ds:
            if "is_partial" in d and d["is_partial"]:
                continue
            #print(' '.join(d["text"]))
            #uast_pprint.pprint(d["code_tree"])
            
            tokens = list(set(list(make_the_tree_good(d["code_tree"]["funcs"]))))
            tokens_total += tokens

    with test:
        
        for d in test:
            if "is_partial" in d and d["is_partial"]:
                continue
            #print(' '.join(d["text"]))
            #uast_pprint.pprint(d["code_tree"])
            
            tokens = list(set(list(make_the_tree_good(d["code_tree"]["funcs"]))))
            tokens_total += tokens

    tokens_total = list(set(tokens_total))
    print(tokens_total) 
    print(len(tokens_total)) #1850

    with open('tokens.txt', 'w') as f:
        for token in tokens_total:
            f.write("%s\n" % token)

def get_tokens():
    with open('tokens.txt', 'r') as f:
        tokens = f.read().splitlines()
        return tokens

def generate_output(tokens, unique):
    values = np.zeros((len(tokens), len(unique)))
    for i in range(len(tokens)):
        index = unique.index(str(tokens[i]))
        values[i][index] = 1
    return values

def generate_tokens(output, unique):
    output = np.array(output)
    tokens = []
    for out in output:
        idx = np.argmax(out)
        tokens.append(unique[idx])
    return tokens

def even_embeddings(embed, n):
    if len(embed) < n:
        diff =  n - len(embed)
        embed = np.vstack([embed, np.zeros((diff,len(embed[0])))])
    return embed

def even_tokens(tokens, n):
    tokens = np.concatenate((['START'],tokens,['END'] ))
    if len(tokens) < n:
        diff =  n - len(tokens)
        tokens = np.concatenate((tokens, np.full(diff, '')))
    return tokens

In [4]:
def get_embeddings(text, preprocessor, encoder):
  sentences_dirty = text.split('.')

  max_len = 0

  sentences = []
  for sent in sentences_dirty:
    sent = re.sub('[,;-]', '', sent)
    if len(sent) > 0:
        sentences.append(sent)
        splitted = sent.split()
        if len(splitted) > max_len:
            max_len = len(splitted)

  encoder_inputs = preprocessor(sentences)
  outputs = encoder(encoder_inputs)

  x = outputs["sequence_output"].numpy()

  return x

In [9]:
_, _, test = read_naps_dataset()
X_train = []
Y_train = []
X_test = []
Y_test = []

Y_tokens = []
Y_tokens2 = []
max_embs_len = 0
max_tokens_len = 0
counter = 0
stopper = 100
stopper2 = 1
# with ds:
#     for d in ds:
#         if "is_partial" in d and d["is_partial"]:
#             continue
#         counter += 1
#         masked_embs = get_embeddings(' '.join(d["text"]), preprocessor, encoder)
#         if len(masked_embs) > max_embs_len:
#             max_embs_len = len(masked_embs)
#         X_train.append(masked_embs)    
#         tokens = make_the_tree_good(d["code_tree"]["funcs"])
#         #print(make_the_tree_good(d["code_tree"]["funcs"]))
#         if len(tokens) > max_tokens_len:
#             max_tokens_len = len(tokens)
#         Y_tokens.append(tokens)
#         if counter == stopper:
#             break
counter = 0
with test:
    for t in test:
        if "is_partial" in t and t["is_partial"]:
            continue
        counter += 1
        masked_embs = get_embeddings(' '.join(t["text"]), preprocessor, encoder)
        if len(masked_embs) > max_embs_len:
            max_embs_len = len(masked_embs)
        X_test.append(masked_embs)    
        tokens = list(flatten(t["code_tree"]["funcs"]))
        #print(make_the_tree_good(d["code_tree"]["funcs"]))
        if len(tokens) > max_tokens_len:
            max_tokens_len = len(tokens)
        Y_tokens2.append(tokens)
        if counter == stopper2:
            break

In [7]:
def get_BERT_x(X):
  new_X = []
  for ex in X:
    if ex.shape[0] < 15:
      diff =  15 - len(ex)
      embed = np.vstack([ex, np.zeros((diff, 128, 768))])
      new_X.append(embed)
    elif ex.shape[0] == 15:
      new_X.append(ex)
    else:
      continue
  return np.array(new_X)

In [12]:
X_train = get_BERT_x(X_train)
X_train = X_train.reshape(X_train.shape[0],1920,768)

In [13]:
with open('X_train.npy', 'wb') as f:
    np.save(f, X_train)

In [10]:
X_test = get_BERT_x(X_test)
X_test = X_test.reshape(X_test.shape[0],1920,768)

In [14]:
with open('X_test.npy', 'wb') as f:
    np.save(f, X_test)

In [15]:
unique = get_tokens()

In [16]:
Y_train = []
max = 701
for ex in Y_tokens:
    #print(even_tokens(ex,max_tokens_len))
    Y_train.append(generate_output(even_tokens(np.array(ex),max), unique))

In [17]:
Y_train = np.array(Y_train)

In [18]:
with open('Y_train.npy', 'wb') as f:
    np.save(f, Y_train)

In [19]:
Y_test = []
max = 701
for ex in Y_tokens2:
    #print(even_tokens(ex,max_tokens_len))
    Y_test.append(generate_output(even_tokens(np.array(ex),max), unique))

In [21]:
Y_test = np.array(Y_test)
with open('Y_test.npy', 'wb') as f:
    np.save(f, Y_test)

In [22]:
Y_test.shape

(1, 701, 1850)